In [2]:
############ Generate Images ###################
import math
import numpy as np
import cv2
import cv2, os
import matplotlib.pyplot as plt
from tqdm import tqdm
import fnmatch

In [3]:
######## TEMPORAL: read prediciton, ground truth and  mouse (x,y) from CSV ################

pred_x_data = np.genfromtxt('/home/kkjk/LSTM/Co-ordinates_mouse_cursor_GT_prediction/prediction_XX_26180.csv').astype(np.int64)
pred_y_data = np.genfromtxt('/home/kkjk/LSTM/Co-ordinates_mouse_cursor_GT_prediction/prediction_YY_26180.csv').astype(np.int64)

gt_x_data = np.genfromtxt('/home/kkjk/LSTM/Co-ordinates_mouse_cursor_GT_prediction/GT_Fix_XX_26180.csv').astype(np.int64)
gt_y_data = np.genfromtxt('/home/kkjk/LSTM/Co-ordinates_mouse_cursor_GT_prediction/GT_Fix_YY_26180.csv').astype(np.int64)

mouse_x_data = np.genfromtxt('/home/kkjk/LSTM/Co-ordinates_mouse_cursor_GT_prediction/mouse_XX_26180.csv').astype(np.int64)
mouse_y_data = np.genfromtxt('/home/kkjk/LSTM/Co-ordinates_mouse_cursor_GT_prediction/mouse_YY_26180.csv').astype(np.int64)




In [4]:
################## TEMPORAL ############################## 

def GenerateImages(Input, Imagename):
    """ 
    This function generates binary map/image of size 1200*1920 out of (x,y)co-ordinates
    
    Parameters:
        Input(stacked array of (x,y)): join (x,y) of prediction or ground truth or mouse
        Imagename(string): Name of the image to be created and saved
    
    Returns:
        path of created image. changes for spatial and temporal data. 
    """
    Op_Image = np.zeros((1200, 1920), dtype=np.int64)

    for x, y in zip(Input[0], Input[1]):
        # print('x,y', x, y)
        Op_Image[y][x] = 255
        op_map = np.float32(Op_Image)
    NewImage = '/home/kkjk/LSTM/Images_to_Evaluate/Temporal_Images/'+Imagename
    cv2.imwrite(NewImage, op_map)
    return NewImage


In [5]:
## 2 test users with total of 38 interfaces, so 38*340 = 12920 (x,y) points
## Temporal attention prediction evaluates one point per user and per task, creating 12920 images and heatmaps

k = 1

for i in range(13260,13261,1):

    # print(i,i+340)
    pred_X_Y = np.vstack((pred_x_data[i], pred_y_data[i].T))
    gt_X_Y = np.vstack((gt_x_data[i], gt_y_data[i].T))
    mouse_X_Y = np.vstack((mouse_x_data[i], mouse_y_data[i].T))
    Pred_X = GenerateImages(pred_X_Y, 'predict_temporal_'+str(i)+'.png')
    True_X = GenerateImages(gt_X_Y, 'true_temporal_'+str(i)+'.png')
    Mouse_X = GenerateImages(mouse_X_Y, 'mouse_temporal_'+str(i)+'.png')
    k = k + 1
    print(k)
print(k, 'individual temporal images generated')

2
2 individual temporal images generated


In [6]:
#################### GENERATING HEATMAP: SPATIAL ###########################
def GaussianMask(sizex, sizey, sigma, center=None, fix=1):
    """
    This function performs gaussian convolution/masking
    Parameters:
        sizex  : mask width
        sizey  : mask height
        sigma  : gaussian standard deviation
        center : gaussian mean
        fix    : gaussian max
    returns:
        gaussian mask
    """

    x = np.arange(0, sizex, 1, float)
    y = np.arange(0, sizey, 1, float)
    x, y = np.meshgrid(x, y)

    if center is None:
        x0 = sizex // 2
        y0 = sizey // 2

    else:
        if np.isnan(center[0]) == False and np.isnan(center[1]) == False:
            x0 = center[0]
            y0 = center[1]

        else:
            return np.zeros((sizey, sizex))

    return fix * np.exp(-4 * np.log(2) * ((x - x0) ** 2 + (y - y0) ** 2) / sigma ** 2)


def Fixpos2Densemap(fix_arr, width, height): #, imgfile, alpha=0.5, threshold=10):
    """
    This function creates heatmap/attention map
    Parameters:
        fix_arr   : fixation array number of subjects x 3(x,y,fixation)
        width     : output image width
        height    : output image height
        imgfile   : image file (optional)
        alpha     : marge rate imgfile and heatmap (optional)
        threshold : heatmap threshold(0~255)
    returns:
        heatmap
    """
    W = width
    H = height
    heatmap = np.zeros((H, W), np.float32)
    for n_subject in tqdm(range(fix_arr.shape[0])):
        heatmap += GaussianMask(W, H, 64, (fix_arr[n_subject, 0], fix_arr[n_subject, 1]), fix_arr[n_subject, 2])

    # Normalization to have values between 0-1
    
    heatmap = heatmap / np.amax(heatmap)
    heatmap = heatmap * 255
    heatmap = heatmap.astype("uint8")
    # heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    return heatmap

    if imgfile.any():
    # Resize heatmap to imgfile shape 
        h, w, _ = imgfile.shape
        heatmap = cv2.resize(heatmap, (w, h))
        heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
      
    # Create mask
        mask = np.where(heatmap<=threshold, 1, 0)
        mask = np.reshape(mask, (h, w, 1))
        mask = np.repeat(mask, 3, axis=2)
      
    # Marge images
        marge = imgfile*mask + heatmap_color*(1-mask)
        marge = marge.astype("uint8")
        marge = cv2.addWeighted(imgfile, 1-alpha, marge,alpha,0)
        print('Im with screenshot and color')

        return marge, mask, heatmap_color

    else:
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        return heatmap

In [9]:
# imdir for reading input and outdir for storing output heatmap




In [8]:
##### Generate heatmaps for prediction data ################
p_imdir = '/home/kkjk/LSTM/Images_to_Evaluate/Temporal_Images/tem_prediction/'

p_outdir =  '/home/kkjk/LSTM/Images_to_Evaluate/Temporal_heatmaps/tem_prediction_heatmaps/'

In [14]:
for i in range(13260,26180,1):

    pfile = 'predict_temporal_'+str(i)+'.png'
  
    
    ppath = os.path.join(p_imdir, pfile)
    print('prediction:',ppath)
      
    img = cv2.imread(ppath, cv2.IMREAD_GRAYSCALE)
    result = np.where(img == 255)
#     print('result',result)
    # y = np.array(result[0])[::-1]
    y = np.array(result[0])
    x = np.array(result[1])
    z = np.ones(x.shape)
    fix_arr = np.column_stack([x, y, z])
    H, W = img.shape 
    heatmap = Fixpos2Densemap(fix_arr, W, H)#, 0.7, 5)
    print('output', (os.path.join(p_outdir, 'predict_temporal_heatmap_'+str(i)+'.png')))
    cv2.imwrite(os.path.join(p_outdir,'predict_temporal_heatmap_'+str(i)+'.png'), heatmap)
  

100%|██████████| 1/1 [00:00<00:00,  6.12it/s]

prediction: /home/kkjk/LSTM/Images_to_Evaluate/Temporal_Images/tem_prediction/predict_temporal_14751.png
output /home/kkjk/LSTM/Images_to_Evaluate/Temporal_heatmaps/tem_prediction_heatmaps/predict_temporal_heatmap_14751.png


In [11]:
############### Generate heatmap for temporal mouse data ###########

m_imdir = '/home/kkjk/LSTM/Images_to_Evaluate/Temporal_Images/tem_mouse/'
m_outdir = '/home/kkjk/LSTM/Images_to_Evaluate/Temporal_heatmaps/tem_mouse_heatmaps/'


In [12]:
for i in range(13260,26180,340):

    mfile = 'mouse_temporal_'+str(i)+'.png'
   
    mopath = os.path.join(m_imdir, mfile)
    print('mouse:',mopath)

    img = cv2.imread(mopath, cv2.IMREAD_GRAYSCALE)
    result = np.where(img == 255)
    # y = np.array(result[0])[::-1]
    y = np.array(result[0])
    x = np.array(result[1])
    z = np.ones(x.shape)
    fix_arr = np.column_stack([x, y, z])
    H, W = img.shape 
    heatmap = Fixpos2Densemap(fix_arr, W, H)#, 0.7, 5)
    print('output', (os.path.join(m_outdir, mouse_temporal_heatmap_'+str(i)+'.png')))
    cv2.imwrite(os.path.join(m_outdir,mouse_temporal_heatmap_'+str(i)+'.png'), heatmap)

100%|██████████| 1/1 [00:00<00:00,  6.04it/s]

mouse: /home/kkjk/LSTM/Images_to_Evaluate/Temporal_Images/tem_mouse/mouse_temporal_13260.png
output /home/kkjk/LSTM/Images_to_Evaluate/Temporal_heatmaps/tem_mouse_heatmaps/mouse_temporal_13260.png


In [32]:
### EOF ###